## インストール，マウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install gspread

## スプレッドシート定義

In [ ]:
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

def readSheet(id, sheet_name):
  workbook = gc.open_by_key(id)
  worksheet = workbook.worksheet(sheet_name)
  return worksheet.get_all_values()

In [ ]:
ss_id = "スプレッドシートのIDを入力"
ss_aug_id = "スプレッドシートのIDを入力"
ss_single_label_id = "スプレッドシートのIDを入力"

In [ ]:
"""
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("シート2")
print(worksheet.get_all_values())
"""

'\nworkbook = gc.open_by_key(ss_id)\nworksheet = workbook.worksheet("シート2")\nprint(worksheet.get_all_values())\n'

## フォーマット変更

In [ ]:
"""
・5月10日 API get_all_record　がバグっているらしくてget_all_values()を使用
・それに伴いformat_list関数変更
"""
from os import replace
import re
import csv
def format_list(review : list) -> list:
  review_list = []
  patturn = '\d+\.\d\s' #タイトル内　星数取得のためのパターン
  patturn1 = '\d+人のお客様がこれが役に立ったと考えています'
  patturn2 = '人のお客様がこれが役に立ったと考えています'
  del review[0] #['タイトル', 'レビュー', 'カテゴリー', '', '商品名', '', 'URL']　こいつ削除
  for i in range(len(review)):
      dict = {}
      star, useful_num = 0, 0
      useful_text = ''
      dict["タイトル"] = review[i][0]
      dict['タイトル'] = re.sub('5つ星のうち', '', dict['タイトル'])
      star = re.match(patturn, dict['タイトル'])
      dict['星数'] = float(star.group())
      dict['タイトル'] = re.sub(patturn, '', dict['タイトル'])
      dict['レビュー'] = f'{dict["タイトル"]}\n{review[i][1]}'
      dict['カテゴリー'] = review[i][2]
      useful_text = re.search(patturn1, dict['レビュー'])
      if useful_text != None:
        useful_text = useful_text.group()
        dict['レビュー'] = re.sub(useful_text, '', dict['レビュー'])
        useful_num = re.sub(patturn2, '', useful_text)
      else:
        useful_num = 0

      dict['参考になった数'] = int(useful_num)
      review_list.append(dict)
  return review_list

In [ ]:
#拡張データ専用，生成元IDの追加
def aug_format_list(review : list) -> list:
  review_list = []
  patturn = '\d+\.\d\s' #タイトル内　星数取得のためのパターン
  patturn1 = '\d+人のお客様がこれが役に立ったと考えています'
  patturn2 = '人のお客様がこれが役に立ったと考えています'
  del review[0] #['タイトル', 'レビュー', 'カテゴリー', '', '商品名', '', 'URL']　こいつ削除
  for i in range(len(review)):
    try:
      dict = {}
      star, useful_num = 0, 0
      useful_text = ''
      dict["タイトル"] = review[i][0]
      dict['タイトル'] = re.sub('5つ星のうち', '', dict['タイトル'])
      star = re.match(patturn, dict['タイトル'])
      dict['星数'] = float(star.group())
      dict['タイトル'] = re.sub(patturn, '', dict['タイトル'])
      dict['レビュー'] = f'{dict["タイトル"]}\n{review[i][1]}'
      dict['カテゴリー'] = review[i][2]
      useful_text = re.search(patturn1, dict['レビュー'])
      if useful_text != None:
        useful_text = useful_text.group()
        dict['レビュー'] = re.sub(useful_text, '', dict['レビュー'])
        useful_num = re.sub(patturn2, '', useful_text)
      else:
        useful_num = 0

      dict['参考になった数'] = int(useful_num)
      dict['生成元ID'] = int(review[i][3])
      review_list.append(dict)
    except Exception as e:
      # その他のエラーをキャッチ
      print(f"Error processing review {i}: {e}")
  return review_list

## シートごとに変数代入

In [ ]:
sheet2_list = format_list(readSheet(ss_id,"シート2"))
sheet3_list = format_list(readSheet(ss_id,"シート3"))
sheet4_list = format_list(readSheet(ss_id,"シート4"))
sheet5_list = format_list(readSheet(ss_id,"シート5"))
sheet6_list = format_list(readSheet(ss_id,"シート6"))
sheet7_list = format_list(readSheet(ss_id,"シート7"))
sheet8_list = format_list(readSheet(ss_id,"シート8"))
sheet9_list = format_list(readSheet(ss_id,"シート9"))
all_list = sheet2_list + sheet3_list + sheet4_list + sheet5_list + sheet6_list + sheet7_list + sheet8_list + sheet9_list
#all_list = format_list(all_list)


In [ ]:
#オーグメンテーション用
aug_list = readSheet(ss_aug_id,"シート4")
aug_list = format_list(aug_list)

In [ ]:
#オーグメンテーション実験用
aug_list_label2_method1 = readSheet(ss_aug_id,"ラベル2-手法1")
aug_list_label2_method2 = readSheet(ss_aug_id,"ラベル2-手法2")
aug_list_label2_method3 = readSheet(ss_aug_id,"ラベル2-手法3")

aug_list_label5_method1 = readSheet(ss_aug_id,"ラベル5-手法1")
#aug_list_label5_method2 = readSheet(ss_aug_id,"ラベル5-手法2")
aug_list_label5_method3 = readSheet(ss_aug_id,"ラベル5-手法3")
aug_list_label6_method1 = readSheet(ss_aug_id,"ラベル6-手法1")
#aug_list_label6_method2 = readSheet(ss_aug_id,"ラベル6-手法2")
aug_list_label6_method3 = readSheet(ss_aug_id,"ラベル6-手法3")
aug_list_label7_method1 = readSheet(ss_aug_id,"ラベル7-手法1")
#aug_list_label7_method2 = readSheet(ss_aug_id,"ラベル7-手法2")
aug_list_label7_method3 = readSheet(ss_aug_id,"ラベル7-手法3")

aug_list_label2_method1 = aug_format_list(aug_list_label2_method1)
aug_list_label2_method2 = aug_format_list(aug_list_label2_method2)
aug_list_label2_method3 = aug_format_list(aug_list_label2_method3)

aug_list_label5_method1 = aug_format_list(aug_list_label5_method1)
#aug_list_label5_method2 = aug_format_list(aug_list_label5_method2)
aug_list_label5_method3 = aug_format_list(aug_list_label5_method3)
aug_list_label6_method1 = aug_format_list(aug_list_label6_method1)
#aug_list_label6_method2 = aug_format_list(aug_list_label6_method2)
aug_list_label6_method3 = aug_format_list(aug_list_label6_method3)
aug_list_label7_method1 = aug_format_list(aug_list_label7_method1)
#aug_list_label7_method2 = aug_format_list(aug_list_label7_method2)
aug_list_label7_method3 = aug_format_list(aug_list_label7_method3)


In [ ]:
"""
#シングルラベルレビュー用
single_label_list = readSheet(ss_single_label_id,"シート1")
single_label_list = format_list(single_label_list)
"""

'\n#シングルラベルレビュー用\nsingle_label_list = readSheet(ss_single_label_id,"シート1")\nsingle_label_list = format_list(single_label_list)\n'

In [ ]:
len(all_list)

766

In [ ]:
"""
#pattrunマッチ確認用
patturn1 = '\d人のお客様がこれが役に立ったと考えています'
text = '寝室に置いておくのに丁度良いサイズ\n2人のお客様がこれが役に立ったと考えています'
usefull_text = re.search(patturn1, text)
print(usefull_text)
"""

"\n#pattrunマッチ確認用\npatturn1 = '\\d人のお客様がこれが役に立ったと考えています'\ntext = '寝室に置いておくのに丁度良いサイズ\n2人のお客様がこれが役に立ったと考えています'\nusefull_text = re.search(patturn1, text)\nprint(usefull_text)\n"

## csvファイル書き込み

In [ ]:
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/review/"
# CSVファイル名
csv_name = 'review_ver20250108.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(all_list)

In [ ]:
#オーグメンテーション用
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/review/"
# CSVファイル名
csv_name = 'aug_review_ver20241128.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数", "生成元ID"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list)

# -----------手法1-----------
csv_name = 'aug_review_label2-m1.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label2_method1)

csv_name = 'aug_review_label5-m1.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label5_method1)

csv_name = 'aug_review_label6-m1.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label6_method1)

csv_name = 'aug_review_label7-m1.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label7_method1)
"""
"""
# -----------手法2-----------
csv_name = 'aug_review_label2-m2.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label2_method2)
"""
csv_name = 'aug_review_label5-m2.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label5_method2)

csv_name = 'aug_review_label6-m2.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label6_method2)

csv_name = 'aug_review_label7-m2.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label7_method2)
"""
# -----------手法3-----------
csv_name = 'aug_review_label2-m3.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label2_method3)

csv_name = 'aug_review_label5-m3.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label5_method3)

csv_name = 'aug_review_label6-m3.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label6_method3)

csv_name = 'aug_review_label7-m3.csv'
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list_label7_method3)
"""
"""

'\n'

In [ ]:
"""
#シングルラベルレビュー用
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/"
# CSVファイル名
csv_name = 'single_label_review.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(single_label_list)
"""

'\n#シングルラベルレビュー用\n# CSVファイルの保管先\ncsv_path = "/content/drive/MyDrive/"\n# CSVファイル名\ncsv_name = \'single_label_review.csv\'\nfield_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]\n# リストをCSVファイルに出力（\'w\'の場合は、新規作成）\nwith open(csv_path+csv_name, \'w\', newline=\'\') as f:\n    writer = csv.DictWriter(f, fieldnames = field_name)\n    writer.writeheader()\n    writer.writerows(single_label_list)\n'